# 1. Data Preprocessing:
*필요 library:
    seaborn, tensorflow, pytorch, torchvision, imbalanced-learn
    
### 1) Getting data

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os, glob

In [ ]:
#파일 경로 가져오기
PATH = './train/'

image_dirs = os.listdir(PATH)
image_paths = dict()
for image_dir in tqdm(image_dirs):
    try:
        files = glob.glob(PATH + image_dir+"/*.png") 
        image_paths[image_dir] = [file for file in files]
    except:
        FileNotFoundError

In [ ]:
image_paths

In [ ]:
#이미지 파일 & Label & filename & path 가져오기
import PIL
from PIL import Image
import torch
import torchvision
from torch.utils.data import  TensorDataset, DataLoader
import torchvision.transforms as transforms
from keras.preprocessing.image import img_to_array

#image transform
transform = transforms.Compose([
    transforms.Resize([512,512]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

data = dict()
data['label'] = []
data['filename'] = []
data['image_arr'] = []
for label,paths in tqdm(image_paths.items()):
    for path in paths:
        img = Image.open(path)
        img = img.convert('RGB')
        data['filename'].append(path)
        img_arr = img_to_array(transform(img)) #3,480,480
        img_arr = np.transpose(img_arr, (2,1,0)) # 480,480,3
        data['image_arr'].append(img_arr)
        data['label'].append(label)

In [ ]:
data['filename']

In [ ]:
data["label"][0]

In [ ]:
#파일 정보 확인 및 이미지 확인, data imbalance 확인 
from collections import Counter
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array,load_img
print('number of samples: ', len(data['image_arr']))
print('dict keys: ', list(data.keys()))
print('image shape: ', data['image_arr'][0].shape)
count_label = Counter(data['label'])
print('Original dataset shape')
print(sorted(count_label.items()))

#show images
fig, axes = plt.subplots(1, len(count_label.items()))
fig.set_size_inches(15,4)
fig.tight_layout()
 
for ax, label in zip(axes, np.unique(data['label'])):
    idx = data['label'].index(label)
    ax.imshow(array_to_img(data['image_arr'][idx]))
    ax.axis('off')
    ax.set_title(label)

In [ ]:
#imbalance 시각화
import seaborn as sns

plt.figure(figsize=(10,5))
sns.countplot(data['label'])
plt.show()

### 2) Data Augmentation
* 왜곡된 image를 가장 많은 label 수에 맞추어 (0 SPOT, 1140개) directory에 바로 생성.

In [ ]:
Counter(data['label'])

In [ ]:
for img,label in zip(data['image_arr'],data['label']):
    print(label)
    print("먼데")

In [ ]:
save_to_dir=f"{PATH}{label}/"
save_to_dir

In [ ]:
#generator 생성
data_gen = ImageDataGenerator(rotation_range=[0, 45], 
                               width_shift_range= [0, 0.1], 
                               height_shift_range= [0, 0.1],
                               rescale = 1.0/255, #rgb값을 normalize, 학습 효율적
                               shear_range=0.1, #distort
                               zoom_range = [0, 0.1],
                               horizontal_flip = True,
                               vertical_flip= True,
                               fill_mode='nearest'
                              )

label_count = Counter(data['label'])
label_max = max(label_count.values()) # 220

i=0
for img,label in zip(data['image_arr'],data['label']):
    x = img
    x = x.reshape((1,)+x.shape)
    img_count = label_count[label] #현재 label을 가진 사진 수
    if img_count < label_max:
        i = label_max-img_count #해당 label에 대해 추가로 생성해야 하는 사진 수 (가장 많은 label의 사진 수에서 뺀 값)
        i = i//img_count #이 사진으로 만들어야 하는 보정 사진 수
        for batch in data_gen.flow(x,
                                 batch_size=1,
                                 save_to_dir=f"{PATH}{label}/", #기존 dir에 생성.
                                 save_prefix=f'made_'+label[0],
                                 save_format='jpg'):
            i-=1
            if i<0:
                break

In [2]:
from PIL import Image
import torchvision.transforms as transforms


transform = transforms.Compose([
    transforms.ToTensor(), #이 과정에서 [0, 255]의 범위를 갖는 값들을 [0.0, 1.0]으로 정규화 
])


# image = Image.open('1차 용접부 데이터(추가데이터)_최종/test/1/7.jpg').convert('RGB')
image = Image.open('./train/bottle-broken_large/10204.png')
image = transform(image).unsqueeze(0)
image = image.numpy()
image.save("10204test.jpg")



AttributeError: 'Tensor' object has no attribute 'save'